In [2]:
model_path = "yolov7/best.pt"

In [3]:
import sys
sys.path.append("yolov7")


In [4]:
import torch
import numpy as np
from models.experimental import attempt_load
import cv2
from utils.general import non_max_suppression, scale_coords

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:

model = attempt_load("yolov7/best.pt", map_location=device)  # Load model
model.eval()  # Set to evaluation mode

/home/biplop/ai-powered-ecommerce/clothes_detection/yolov7/models/experimental.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(w, map_location=map_lo

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block


/home/biplop/ai-powered-ecommerce/clothes_detection/.conda/lib/python3.8/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Model(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (2): Conv(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (3): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (4): Conv(
      (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
      (act): SiLU(inplace=True)
    )
    (5): Conv(
      (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
      (act): SiLU(inplace=True)
    )
    (6): Conv(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (7): Conv(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), strid

In [12]:
!pwd

/home/biplop/ai-powered-ecommerce/clothes_detection


In [7]:
#importing image 
# img = cv2.imread('tshirt.jpg',0)
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
# img = img / 255.0  # Normalize to [0,1] range
# img = np.transpose(img, (2, 0, 1))  # Change shape from (H, W, C) to (C, H, W)
# img = np.expand_dims(img, axis=0)  # Add batch dimension
# img = torch.tensor(img, dtype=torch.float32).to(device)

# Load and preprocess the image
img = cv2.imread('1538.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
img0 = img.copy()  # Keep a copy of the original image for visualization
img = cv2.resize(img, (640, 640))
img = img / 255.0  # Normalize to [0,1] range
img = np.transpose(img, (2, 0, 1))  # Change shape from (H, W, C) to (C, H, W)
img = np.expand_dims(img, axis=0)  # Add batch dimension
img = torch.tensor(img, dtype=torch.float32).to(device)

In [8]:
with torch.no_grad():
    pred = model(img)[0]
    pred = non_max_suppression(pred, conf_thres=0.5, iou_thres=0.6)

In [15]:
print("Input image shape:", img.shape)

Input image shape: torch.Size([1, 3, 640, 640])


In [9]:
# Process detections
detections = []
for i, det in enumerate(pred):  # detections per image
    if len(det):
        # Rescale boxes from img_size to img0 size
        det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()
        detections.append(det.cpu().numpy())
print(detections)        

# detections now contains the bounding boxes of detected clothes

[array([[        100,         744,        1800,        2392,     0.94968,           0]], dtype=float32)]


In [17]:
# Crop detected regions
cropped_images = []
for det in detections[0]:
    x1, y1, x2, y2, conf, cls = det
    cropped_img = img0[int(y1):int(y2), int(x1):int(x2)]
    cropped_images.append(cropped_img)

In [ ]:
#to visualize 
if len(cropped_images) == 0:
    print("No detections to visualize!")
else:
    for i, cropped_img in enumerate(cropped_images):
        # Convert from RGB to BGR (OpenCV uses BGR format)
        cropped_img_bgr = cv2.cvtColor(cropped_img, cv2.COLOR_RGB2BGR)
        cv2.imshow(f"Cropped Clothes {i+1}", cropped_img_bgr)
        cv2.waitKey(0)  # Press any key to close the window
        cv2.destroyAllWindows()

In [54]:
import clip
from PIL import Image

# Load CLIP model
clip_model, preprocess = clip.load("ViT-B/32", device=device)

# Extract feature vectors for each cropped image
feature_vectors = []
for cropped_img in cropped_images:
    # Convert to PIL image
    pil_img = Image.fromarray(cropped_img)
    
    # Preprocess the image for CLIP
    preprocessed_img = preprocess(pil_img).unsqueeze(0).to(device)
    
    # Extract feature vector
    with torch.no_grad():
        image_features = clip_model.encode_image(preprocessed_img)
    
    feature_vectors.append(image_features.cpu().numpy())

# feature_vectors now contains the feature vectors for each cropped image

ModuleNotFoundError: No module named 'clip'

In [53]:
!pip install git+https://github.com/openai/CLIP.git